https://stackoverflow.com/questions/33642158/what-is-the-difference-between-cross-val-score-with-scoring-roc-auc-and-roc-au

In [28]:
from sklearn.datasets import load_digits
from sklearn import linear_model, svm
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score, StratifiedKFold 
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.metrics import roc_auc_score

In [2]:
# Let's use the Digit dataset
digits = load_digits(n_class=4)
X,y = digits.data, digits.target
y[y==2] = 0 # Increase problem dificulty
y[y==3] = 1 # even more

In [69]:
LR = LogisticRegression()
SVM = svm.LinearSVC()
fourfold = StratifiedKFold(y, n_folds=4, random_state=4)
gs = GridSearchCV(LR, param_grid={}, cv=fourfold, scoring='roc_auc', return_train_score=True)
gs.fit(X,y)
gs_scores = np.array([gs.cv_results_['split0_test_score'][0],
                      gs.cv_results_['split1_test_score'][0],
                      gs.cv_results_['split2_test_score'][0],
                      gs.cv_results_['split3_test_score'][0]])
gs_scores

array([ 0.99865657,  0.99876528,  0.99678973,  0.99837662])

In [70]:
cv_scores = cross_val_score(LR, X, y, cv=fourfold, scoring='roc_auc')
cv_scores

array([ 0.99865657,  0.99876528,  0.99678973,  0.99837662])

In [71]:
loop_scores = list()
for idx_train, idx_test in fourfold:
    X_train, y_train, X_test, y_test = X[idx_train], y[idx_train], X[idx_test], y[idx_test]
    LR.fit(X_train, y_train)
    y_prob = LR.predict_proba(X_test)
    auc = roc_auc_score(y_test, y_prob[:,1])
    loop_scores.append(auc)
loop_scores

[0.99865657059110891,
 0.99876527966415607,
 0.9967897271268058,
 0.99837662337662336]

In [73]:
print ([((a==b) and (b==c)) for a,b,c in zip(gs_scores,cv_scores,loop_scores)])

[True, True, True, True]


In [77]:
auc_list = []
for idx_train, idx_test in fourfold:
    X_train, y_train, X_test, y_test = X[idx_train], y[idx_train], X[idx_test], y[idx_test]
    SVM.fit(X_train, y_train)
    y_prob = SVM.decision_function(X_test)
    prob_pos = (y_prob - y_prob.min()) / (y_prob.max() - y_prob.min())
    auc = roc_auc_score(y_test, prob_pos)
    auc_list.append(auc)
auc_list

[0.99890083048363465,
 0.99876527966415607,
 0.99506111865662439,
 0.99862637362637363]